In [1]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')



Mounted at /content/drive/


In [2]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/cs532/cnn-pruning-status_200.zip", 'r')
zip_ref.extractall("content/tmp")
zip_ref.close()

In [3]:
!pwd

/content


In [4]:
!pip uninstall torch

Found existing installation: torch 1.10.0+cu111
Uninstalling torch-1.10.0+cu111:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.7/dist-packages/caffe2/*
    /usr/local/lib/python3.7/dist-packages/torch-1.10.0+cu111.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch/*
Proceed (y/n)? ERROR: Operation cancelled by user


In [5]:
!curl -s https://course.fast.ai/setup/colab | bash

bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE html>'


In [6]:
FOLDERNAME = 'content/tmp/cnn-pruning-status_200'
%cd $FOLDERNAME

/content/content/tmp/cnn-pruning-status_200


In [7]:
import torch
import copy
import torch.nn.utils.prune as prune
import math
from torchvision import datasets,transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

In [8]:
# Load the pretrained model
from cifar10_models.resnet import resnet18
my_model = resnet18()

# Pretrained model
my_model = resnet18(pretrained=True)
my_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
def calculate_sparsity(model):
  total_count = 0
  zero_count = 0
  
  for buffer_name, buffer in model.named_buffers():
    zero_count += torch.sum(buffer == 0).item()
    if zero_count>0:
      total_count += buffer.nelement()

  print("Total params: ", total_count)
  print("Zero params: ", zero_count)
  return (math.ceil(zero_count*100/total_count))

In [10]:
print("Preparing_data ======>")
transform_train = transforms.Compose([
                                    transforms.RandomCrop(32,padding=4),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))
])
                                
trainset = torchvision.datasets.CIFAR10(
    root = './data',train=True,download=True,transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=128,shuffle=True,num_workers=2)

Preparing_data ======>


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [12]:
loss_history = []
correct_history = []
val_loss_history = []
criterion = nn.CrossEntropyLoss()

In [13]:
def get_optimizer(model):
  optimizer = optim.SGD(model.parameters(), lr=0.1,momentum=0,weight_decay=5e-4)
  return optimizer

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [15]:
def train(epoch,model):
  model.train()
  train_acc = 0
  train_loss = 0
  correct = 0
  total = 0

  for batch_idx, (inputs, targets) in enumerate(trainloader):
    inputs,targets = inputs.to(device),targets.to(device)
    outputs = model(inputs)
    loss = criterion(outputs,targets)
    optimizer = get_optimizer(model)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    _,predicted = outputs.max(1)
    total +=targets.size(0)
    correct += predicted.eq(targets).sum().item()
    train_acc = 100 *correct/total
  if epoch%5==1:
    print("Training")
    print('[%d/%d] Loss: %.3f | Acc: %.3f (%d/%d)' %(batch_idx,len(trainloader),train_loss/(batch_idx+1),100.*correct/total,correct,total))
    print('Current learning rate is: {}'.format(optimizer.param_groups[0]['lr']))
  loss_history.append(train_loss)
  correct_history.append(train_acc)

  return min(loss_history),max(correct_history)

In [17]:
# Build and save the oneshot pruning model with 50 % sparsity ratio
model1 = copy.deepcopy(my_model)
parameters_to_prune =[]
for module_name, module in model1.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, "weight"))
print("Layers to prune {}".format((len(parameters_to_prune))))

prune.global_unstructured(
parameters_to_prune,
pruning_method=prune.L1Unstructured,
amount=0.5,
)
print("Sparsity of model1", calculate_sparsity(model1))


Layers to prune 21
Total params:  11173972
Zero params:  5582176
Sparsity of model1 50


In [18]:
# Retrain the model for 100 epochs
model1 = model1.to(device)
epochs = 51
for e in range(epochs):
  if (e+1)%5==1:
    print(f"Epoch ========> {e+1}")
  train(e+1,model1)
  torch.save(model1, "cifar10_models/state_dicts/one_shot_50.pt")

Epoch ========> 1
Training
[390/391] Loss: 1.623 | Acc: 40.194 (20097/50000)
Current learning rate is: 0.1
Epoch ========> 6
Training
[390/391] Loss: 0.611 | Acc: 79.200 (39600/50000)
Current learning rate is: 0.1
Epoch ========> 11
Training
[390/391] Loss: 0.434 | Acc: 85.164 (42582/50000)
Current learning rate is: 0.1
Epoch ========> 16
Training
[390/391] Loss: 0.349 | Acc: 88.084 (44042/50000)
Current learning rate is: 0.1
Epoch ========> 21
Training
[390/391] Loss: 0.293 | Acc: 89.902 (44951/50000)
Current learning rate is: 0.1
Epoch ========> 26
Training
[390/391] Loss: 0.258 | Acc: 90.992 (45496/50000)
Current learning rate is: 0.1
Epoch ========> 31
Training
[390/391] Loss: 0.226 | Acc: 92.240 (46120/50000)
Current learning rate is: 0.1
Epoch ========> 36
Training
[390/391] Loss: 0.201 | Acc: 93.132 (46566/50000)
Current learning rate is: 0.1
Epoch ========> 41
Training
[390/391] Loss: 0.181 | Acc: 93.626 (46813/50000)
Current learning rate is: 0.1
Epoch ========> 46
Training
[3

In [16]:
# Build and save the oneshot pruning model with 75 % sparsity ratio
model2 = copy.deepcopy(my_model)
parameters_to_prune =[]
for module_name, module in model2.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, "weight"))
print("Layers to prune {}".format((len(parameters_to_prune))))

prune.global_unstructured(
parameters_to_prune,
pruning_method=prune.L1Unstructured,
amount=0.75,
)
print("Sparsity of model2", calculate_sparsity(model2))

Layers to prune 21
Total params:  11173972
Zero params:  8373264
Sparsity of model2 75


In [18]:
model2 = model2.to(device)
epochs = 51
for e in range(epochs):
  if (e+1)%5==1:
    print(f"Epoch ========> {e+1}")
  train(e+1,model2)
  torch.save(model2, "cifar10_models/state_dicts/one_shot_75.pt")

Epoch ========> 1
Training
[390/391] Loss: 1.003 | Acc: 64.722 (32361/50000)
Current learning rate is: 0.1
Epoch ========> 6
Training
[390/391] Loss: 0.483 | Acc: 83.488 (41744/50000)
Current learning rate is: 0.1
Epoch ========> 11
Training
[390/391] Loss: 0.358 | Acc: 87.584 (43792/50000)
Current learning rate is: 0.1
Epoch ========> 16
Training
[390/391] Loss: 0.296 | Acc: 89.662 (44831/50000)
Current learning rate is: 0.1
Epoch ========> 21
Training
[390/391] Loss: 0.248 | Acc: 91.384 (45692/50000)
Current learning rate is: 0.1
Epoch ========> 26
Training
[390/391] Loss: 0.218 | Acc: 92.502 (46251/50000)
Current learning rate is: 0.1
Epoch ========> 31
Training
[390/391] Loss: 0.194 | Acc: 93.312 (46656/50000)
Current learning rate is: 0.1
Epoch ========> 36
Training
[390/391] Loss: 0.175 | Acc: 93.834 (46917/50000)
Current learning rate is: 0.1
Epoch ========> 41
Training
[390/391] Loss: 0.164 | Acc: 94.198 (47099/50000)
Current learning rate is: 0.1
Epoch ========> 46
Training
[3

In [19]:
# Build and save the oneshot pruning model with 90 % sparsity ratio
model3 = copy.deepcopy(my_model)
parameters_to_prune =[]
for module_name, module in model3.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, "weight"))
print("Layers to prune {}".format((len(parameters_to_prune))))

prune.global_unstructured(
parameters_to_prune,
pruning_method=prune.L1Unstructured,
amount=0.9,
)
print("Sparsity of model3", calculate_sparsity(model3))
torch.save(model3, "cifar10_models/state_dicts/one_shot_90.pt")

Layers to prune 21
Total params:  11173972
Zero params:  10047917
Sparsity of model3 90


In [20]:
model3 = model3.to(device)
epochs = 51
for e in range(epochs):
  if (e+1)%5==1:
    print(f"Epoch ========> {e+1}")
  train(e+1,model3)
  torch.save(model3, "cifar10_models/state_dicts/one_shot_90.pt")

Epoch ========> 1
Training
[390/391] Loss: 1.124 | Acc: 61.032 (30516/50000)
Current learning rate is: 0.1
Epoch ========> 6
Training
[390/391] Loss: 0.275 | Acc: 90.536 (45268/50000)
Current learning rate is: 0.1
Epoch ========> 11
Training
[390/391] Loss: 0.195 | Acc: 93.280 (46640/50000)
Current learning rate is: 0.1
Epoch ========> 16
Training
[390/391] Loss: 0.159 | Acc: 94.604 (47302/50000)
Current learning rate is: 0.1
Epoch ========> 21
Training
[390/391] Loss: 0.139 | Acc: 95.174 (47587/50000)
Current learning rate is: 0.1
Epoch ========> 26
Training
[390/391] Loss: 0.126 | Acc: 95.668 (47834/50000)
Current learning rate is: 0.1
Epoch ========> 31
Training
[390/391] Loss: 0.117 | Acc: 95.988 (47994/50000)
Current learning rate is: 0.1
Epoch ========> 36
Training
[390/391] Loss: 0.109 | Acc: 96.222 (48111/50000)
Current learning rate is: 0.1
Epoch ========> 41
Training
[390/391] Loss: 0.100 | Acc: 96.542 (48271/50000)
Current learning rate is: 0.1
Epoch ========> 46
Training
[3